In [4]:
import numpy as np
import torch

In [5]:
device = torch.device("cuda:0")

In [307]:
characters = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
chars_len = len(characters)
seq_len = 50

def index_to_char(ind):
    return characters[ind]

def char_to_index(char):
    return characters.index(char)

def to_sequence(string):
    if len(string) < seq_len:
        string += ' ' * (seq_len - len(string))
    elif len(string) > seq_len:
        string = string[len(string) - seq_len:]
    ids = [char_to_index(ch) for ch in string.lower()]
    return ids
    
def process_dataset(ds):
    return [
        (smpl[0], torch.tensor([[
            [0] * ind + [1] + [0] * (chars_len - ind - 1) for ind in to_sequence(smpl[1])
        ]], dtype=torch.float32)) for smpl in ds
    ]

In [365]:
dataset = [
    ('фюрер вошёл', 'кинуть плотную зигу'),
    ('майн фюрер появился', 'кинуть плотную зигу'),
    ('кто то играет в майнкрафт', 'поиграть вместе'),
    ('печенье лежит', 'съесть'),
    ('свет горит', 'выключить'),
    ('кофе уже готово', 'пить'),
    ('сиденье освободилось', 'сесть'),
    ('игра загрузилась', 'играть'),
    ('наступила полночь', 'спать'),
    ('кто то звонит', 'ответить'),
    ('наступила осень', 'нападать на польшу'),
    ('нашёлся изменник', 'расстрелять'),
    ('сша шутит над японией', 'атомной бомбой'),
    ('наступило лето', 'идти гулять'),
    ('наступил ноябрь', 'начинать челлендж'),
    ('споткнулся', 'подняться'),
    ('вода кипит', 'посолить'),
    ('молоко бежит', 'остановить нахер'),
    ('керас хуже пайторча', 'никак нет'),
    ('стол сломался', 'купить новый'),
    ('растения засыхают', 'полить'),
    ('надо что то проводить', 'репрессии')
]

In [357]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.LSTM(1, chars_len, 1, batch_first=True)
        self.rnn2 = torch.nn.LSTM(chars_len, chars_len, 1, batch_first=True)
        self.rnn3 = torch.nn.LSTM(chars_len, chars_len, 1, batch_first=True)
        self.rnn4 = torch.nn.LSTM(chars_len, chars_len, 1, batch_first=True)
        
    def postprocess(self, x):
        x = torch.argmax(x, dim=2)
        messages = []
        for i in range(x.shape[0]):
            msg_arr = [index_to_char(ind) for ind in x[i].tolist()]
            msg = ''.join(msg_arr).strip()
            messages.append(msg)
        return messages
        
    def forward(self, x, raw=False):        
        if type(x) == str:
            x = [x]
            
        h0 = torch.zeros((1, len(x), chars_len))
        c0 = torch.clone(h0)
        
        x = torch.tensor([
            to_sequence(z) for z in x
        ]).reshape((len(x), seq_len, 1))
    
        x = x.type(torch.float32)
        x, hidden = self.rnn1(x, (h0, c0))
        x, hidden = self.rnn2(x, hidden)
        x, hidden = self.rnn3(x, hidden)
        x, hidden = self.rnn4(x, hidden)

        if raw:
            return x
        
        return self.postprocess(x)

In [366]:
model = Net()

In [367]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [374]:
ds = process_dataset(dataset)

for ep in range(1000):
    for i in range(len(ds)):
        x, y = ds[i]
        pred = model(x, raw=True).type(torch.float32)
        loss = criterion(pred, y)
        print('Epoch', ep, 'Loss:', loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch 0 Loss: tensor(0.0035, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0035, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0031, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0014, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0016, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0021, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0012, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0053, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0030, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0049, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0018, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0022, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0044, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0035, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0022, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0011, grad_fn=<MseLossBackward>)
Epoch 0 Loss: tensor(0.0037, grad_fn=<MseLossBac

Epoch 6 Loss: tensor(0.0025, grad_fn=<MseLossBackward>)
Epoch 6 Loss: tensor(0.0017, grad_fn=<MseLossBackward>)
Epoch 6 Loss: tensor(0.0020, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0033, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0033, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0031, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0013, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0014, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0021, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0012, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0052, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0028, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0049, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0017, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0021, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0044, grad_fn=<MseLossBackward>)
Epoch 7 Loss: tensor(0.0033, grad_fn=<MseLossBac

Epoch 13 Loss: tensor(0.0020, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0044, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0032, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0022, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0009, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0035, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0018, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0024, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0017, grad_fn=<MseLossBackward>)
Epoch 13 Loss: tensor(0.0019, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0036, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0032, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0028, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0012, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0014, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0021, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0012, grad_fn=<MseLossBackward>)
Epoch 14 Loss: tensor(0.0014, g

Epoch 20 Loss: tensor(0.0012, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0013, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0021, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0011, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0014, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0053, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0028, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0046, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0017, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0019, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0044, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0032, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0021, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0008, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0032, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0018, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0024, grad_fn=<MseLossBackward>)
Epoch 20 Loss: tensor(0.0017, g

Epoch 26 Loss: tensor(0.0020, grad_fn=<MseLossBackward>)
Epoch 26 Loss: tensor(0.0007, grad_fn=<MseLossBackward>)
Epoch 26 Loss: tensor(0.0031, grad_fn=<MseLossBackward>)
Epoch 26 Loss: tensor(0.0017, grad_fn=<MseLossBackward>)
Epoch 26 Loss: tensor(0.0023, grad_fn=<MseLossBackward>)
Epoch 26 Loss: tensor(0.0017, grad_fn=<MseLossBackward>)
Epoch 26 Loss: tensor(0.0018, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0028, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0029, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0028, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0012, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0012, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0021, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0011, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0014, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0052, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0027, grad_fn=<MseLossBackward>)
Epoch 27 Loss: tensor(0.0046, g

...

Epoch 970 Loss: tensor(0.0011, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0019, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0004, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0072, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0026, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(3.8463e-05, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0028, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0084, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0026, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0013, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0004, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0004, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0006, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0004, grad_fn=<MseLossBackward>)
Epoch 970 Loss: tensor(0.0019, grad_fn=<MseLossBackward>)
Epoch 970 

Epoch 976 Loss: tensor(9.5750e-05, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(4.4461e-05, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(7.0276e-05, grad_fn=<MseLossBackward>)
Epoch 976 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 977 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 977 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 977 Loss: tensor(7.1860e-05, grad_fn=<MseLossBackward>)
Epoch 977 Loss: tensor(1.5162e-05, grad_fn=<MseLossBackw

Epoch 982 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 982 Loss: tensor(5.2455e-05, grad_fn=<MseLossBackward>)
Epoch 982 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(8.7896e-05, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(8.4176e-05, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(5.7168e-05, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(8.5488e-06, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(0.0006, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(1.1769e-05, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(1.0447e-05, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(7.1579e-05, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(6.7055e-05, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 983 Loss: tensor(7.9917e-05, g

Epoch 989 Loss: tensor(0.0006, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(8.2525e-06, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(9.1859e-06, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(6.3646e-05, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(5.3705e-05, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(6.6364e-05, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(1.8819e-05, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(4.6683e-05, grad_fn=<MseLossBackward>)
Epoch 989 Loss: tensor(0.0001, grad_fn=<MseL

Epoch 995 Loss: tensor(4.5985e-05, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(8.7466e-05, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(9.6678e-05, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(0.0002, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(5.8110e-05, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(9.8425e-05, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(1.6074e-05, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(4.3917e-05, grad_fn=<MseLossBackward>)
Epoch 995 Loss: tensor(0.0001, grad_fn=<MseLossBackward>)
Epoch 996 Loss: tensor(6.7225e-05, grad_fn=<MseLossBackward>)
Epoch 996 Loss: tensor(6.8843e-05, grad_fn=<MseLossBackward>)
Epoch 996 Loss: tensor(5.0062e-05, grad_fn=<MseLossBackward>)
Epoch 996 Loss: tensor(6.3241e-0

ITERATION 1

In [351]:
model([
    'на улице светит солнце',
    'его величество фюрер заходит',
    'наступила ночь',
    'наступило утро',
    'фюрер освободился'
])

['ки аууо   уе', 'играть', 'сиать', 'киать', 'кинуть атттвуть']

ITERATION 2

In [394]:
model([
    'что делать',
    'керас',
    'наступило утро',
    'фюрер пришёл',
    'нужна плотнейшая зига',
    'сша шутит над японией',
    'произошёл непредвиденный случай',
    'что то делать',
    'как правильно кидать зигу',
    'сталин принял решение',
    'на улице ветрено',
    'кофе убежало',
    'молоко бежит',
    'программист уволился'
])

['вопеть',
 'оикак',
 'рападатьять',
 'кинутьеллнодыюю зигу',
 'стоат ь',
 'атомной бомбой',
 'спп   я',
 'водерать',
 'пинауа',
 'вуоитььмлотни д м',
 'попсрть',
 'питт',
 'остановить нахер',
 'супутьеелиндыль']

In [395]:
torch.save(model.state_dict(), 'model.mdl')